## Portfolio Exercise: Starbucks
<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200">
<br>
<br>
 
#### Background Information

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their job candidates. The data for this exercise consists of about 120,000 data points split in a 2:1 ratio among training and test files. In the experiment simulated by the data, an advertising promotion was tested to see if it would bring more customers to purchase a specific product priced at $10. Since it costs the company 0.15 to send out each promotion, it would be best to limit that promotion only to those that are most receptive to the promotion. Each data point includes one column indicating whether or not an individual was sent a promotion for the product, and one column indicating whether or not that individual eventually purchased that product. Each individual also has seven additional features associated with them, which are provided abstractly as V1-V7.

#### Optimization Strategy

Your task is to use the training data to understand what patterns in V1-V7 to indicate that a promotion should be provided to a user. Specifically, your goal is to maximize the following metrics:

* **Incremental Response Rate (IRR)** 

IRR depicts how many more customers purchased the product with the promotion, as compared to if they didn't receive the promotion. Mathematically, it's the ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group (_treatment_) minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group (_control_).

$$ IRR = \frac{purch_{treat}}{cust_{treat}} - \frac{purch_{ctrl}}{cust_{ctrl}} $$


* **Net Incremental Revenue (NIR)**

NIR depicts how much is made (or lost) by sending out the promotion. Mathematically, this is 10 times the total number of purchasers that received the promotion minus 0.15 times the number of promotions sent out, minus 10 times the number of purchasers who were not given the promotion.

$$ NIR = (10\cdot purch_{treat} - 0.15 \cdot cust_{treat}) - 10 \cdot purch_{ctrl}$$

For a full description of what Starbucks provides to candidates see the [instructions available here](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM).

Below you can find the training data provided.  Explore the data and different optimization strategies.

#### How To Test Your Strategy?

When you feel like you have an optimization strategy, complete the `promotion_strategy` function to pass to the `test_results` function.  
From past data, we know there are four possible outomes:

Table of actual promotion vs. predicted promotion customers:  

<table>
<tr><th></th><th colspan = '2'>Actual</th></tr>
<tr><th>Predicted</th><th>Yes</th><th>No</th></tr>
<tr><th>Yes</th><td>I</td><td>II</td></tr>
<tr><th>No</th><td>III</td><td>IV</td></tr>
</table>

The metrics are only being compared for the individuals we predict should obtain the promotion – that is, quadrants I and II.  Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equivalent participants.  

Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 

Get started by reading in the data below.  See how each variable or combination of variables along with a promotion influences the chance of purchasing.  When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final `test_results` function.

In [74]:
# load in packages
from itertools import combinations

from test_results import test_results, score
import numpy as np
import pandas as pd
import scipy as sp
import sklearn as sk
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

# load in the data
train_data = pd.read_csv('./training.csv')
train_data.head()

ID Promotion  purchase  V1         V2        V3  V4  V5  V6  V7
0   1        No         0   2  30.443518 -1.165083   1   1   3   2
1   3        No         0   3  32.159350 -0.645617   2   3   2   2
2   4        No         0   2  30.431659  0.133583   1   1   4   2
3   5        No         0   0  26.588914 -0.212728   2   1   4   2
4   8       Yes         0   3  28.044332 -0.385883   1   1   2   2

1. Inspect Data

In [2]:
train_data.shape

(84534, 10)

In [3]:
train_data.dtypes

ID             int64
Promotion     object
purchase       int64
V1             int64
V2           float64
V3           float64
V4             int64
V5             int64
V6             int64
V7             int64
dtype: object

In [5]:
train_data.isnull().sum()

ID           0
Promotion    0
purchase     0
V1           0
V2           0
V3           0
V4           0
V5           0
V6           0
V7           0
dtype: int64

In [6]:
train_data.describe()

ID      purchase            V1            V2            V3  \
count   84534.000000  84534.000000  84534.000000  84534.000000  84534.000000   
mean    62970.972413      0.012303      1.500662     29.973600      0.000190   
std     36418.440539      0.110234      0.868234      5.010626      1.000485   
min         1.000000      0.000000      0.000000      7.104007     -1.684550   
25%     31467.250000      0.000000      1.000000     26.591501     -0.905350   
50%     62827.500000      0.000000      2.000000     29.979744     -0.039572   
75%     94438.750000      0.000000      2.000000     33.344593      0.826206   
max    126184.000000      1.000000      3.000000     50.375913      1.691984   

                 V4            V5            V6            V7  
count  84534.000000  84534.000000  84534.000000  84534.000000  
mean       1.679608      2.327643      2.502898      1.701694  
std        0.466630      0.841167      1.117349      0.457517  
min        1.000000      1.000000      1.000000      1.000000  
25%        1.000000      2.000000      2.000000      1.000000  
50%        2.000000      2.000000      3.000000      2.000000  
75%        2.000000      3.000000      4.000000      2.000000  
max        2.000000      4.000000      4.000000      2.000000

In [13]:
train_data.groupby(by = ['Promotion', 'purchase'])['ID'].count()

Promotion  purchase
No         0           41851
           1             319
Yes        0           41643
           1             721
Name: ID, dtype: int64

In [14]:
train_data['Promotion'].value_counts()

Yes    42364
No     42170
Name: Promotion, dtype: int64

In [ ]:
train_data.purchase.value_counts()

2. Analyzing Data

In [28]:
# Check significance of invariant metric - number of obs in each group

num_obs_diff = []
n_trials = 1000
n_points = train_data.shape[0]


def cal_diff(data):
    
    return data.query('Promotion == "Yes"').shape[0] - data.query('Promotion == "No"').shape[0]
    

for _ in range(n_trials):
    sample = train_data.sample(n_points, replace=True)
    sample_diff = cal_diff(sample)
    num_obs_diff.append(sample_diff)
    
lower_bound = np.percentile(sample_diff, (1 - .95)/2 * 100)
upper_bound = np.percentile(sample_diff, (1 + .95)/2 * 100)

print("The 97.5% confidence interval is: {}, {}".format(lower_bound, upper_bound))
 

The 97.5% confidence interval is: -186.0, -186.0


In [ ]:
# Plot null hypothesis
null_vals = np.random.normal(0, np.std(num_obs_diff), 10000)

plt.hist(null_vals, bins = 60)
plt.title('Normal Distribution under Null Hypothesis')
plt.axvline(cal_diff(train_data), color = 'r', label = 'Observed Metric')
plt.xlabel('Difference betweem groups')
plt.legend()

In [29]:
p_value = 2 * (null_vals > cal_diff(train_data)).mean()
print('p value is:', p_value)

p value is: 0.5146


Based on the p_value result above, the invariant metric has no significant difference between groups.

There are two evaluation metrics: IRR and NIR. So we need to test each of them at significant level of 0.025. 

In [33]:
# test evaluation metrics irr
irr_diffs = []
nir_diffs = []
 

for _ in range(n_trials):
    sample = train_data.sample(n_points, replace=True)
    irr, nir = score(sample)
    irr_diffs.append(irr)
    nir_diffs.append(nir)

    
irr_null_vals = np.random.normal(0, np.std(irr_diffs), 10000)
nir_null_vals = np.random.normal(0, np.std(nir_diffs), 10000)
irr_p_value = (irr_null_vals > score(train_data)[0]).mean()
nir_p_value = (nir_null_vals > score(train_data)[1]).mean()
print("IRR p value:", irr_p_value)
print("NIR p value:", nir_p_value)

IRR p value: 0.0
NIR p value: 1.0


The results above indicate that IRR is significantly improved with the promotion, but NIR is not. As noted, sending each promotion costs the company 0.15, it would be best to limit that promotion only to those that are most receptive to the promotion. So the next step is to build a supervised learning model to identify which users should be sent with promotion. 

In [39]:
X = train_data.drop(columns = ['ID', 'Promotion', 'purchase'])
Y = train_data['purchase']
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)

In [91]:
# Use Random Forest to train the model 

model = RandomForestClassifier(max_depth=5, class_weight='balanced')
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print(y_pred)
print("f1 score is:", f1_score(y_test, y_pred, average = 'weighted'))

[0 1 1 ..., 1 0 0]
f1 score is: 0.696869230408


In [94]:
def labeling(y):
    y_label = []
    
    for i in y:
        if i == 0:
            y_label.append("No")
        elif i == 1:
            y_label.append("Yes")

    return np.array(y_label)
    


def promotion_strategy(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    y_pred = model.predict(df)
    promotion = labeling(y_pred)
    
    return promotion

In [95]:
# This will test your results, and provide you back some information 
# on how well your promotion_strategy will work in practice

test_results(promotion_strategy)

Nice job!  See how well your strategy worked on our test data below!

Your irr with this strategy is 0.0184.

Your nir with this strategy is 311.80.
We came up with a model with an irr of 0.0188 and an nir of 189.45 on the test set.

 How did you do?


(0.018398292606393869, 311.79999999999995)